In [0]:
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import glob

# Make sure you have access to the images' folders
drive.mount('/content/drive/')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd drive/Team Drives

/content/drive/Team Drives


In [0]:
cd ECENproject2

/content/drive/Team Drives/ECENproject2


In [0]:
#### Importing the training images  ####
# This needs Drupad's script for corroboration
#--------------------------------------#
#images = []
#i = 0

#images_list = glob.glob("stitched_pics/*.jpg")
#images = np.array([np.array(Image.open(name)) for name in images_list])

# normalizing
#images =/ 255

# !!!! STILL NEED LABELS' VALUES !!!!
#X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.25)

In [0]:
####The Convolutional Neural Network for stitched images####
#----------------------------------------------------------#

# Still requires tweaking. Also keep it simple for faster forward\backward propagations
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape= (360, 320, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [0]:
####       Running the CNN          ####
#--------------------------------------#

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

#history = model.fit(X_train, y_train, epochs = 30, batch_size = 200, verbose = 2)

#loss  = model.evaluate(X_test, y_test)
#predictions = model.predict(X_test)

#model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 358, 318, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 179, 159, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 177, 157, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 88, 78, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 86, 76, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 43, 38, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 39, 34, 64)        102464    
__________

In [0]:
nb_epochs = 10
batch_size = 15
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    validation_split=0.025) # set validation split

train_generator = train_datagen.flow_from_directory(
    'stitched_pic_with_far_objs',
    target_size=(360, 320),#width , height 
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    'stitched_pic_with_far_objs', # same directory as training data
    target_size=(360, 320),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

model.save("stitched_result_5.h5")

Found 2709 images belonging to 2 classes.
Found 69 images belonging to 2 classes.
Epoch 1/10
180/180 [==============================] - 1446s 8s/step - loss: 0.6767 - acc: 0.5715 - val_loss: 0.5016 - val_acc: 0.8000
Epoch 2/10
180/180 [==============================] - 85s 471ms/step - loss: 0.2771 - acc: 0.8889 - val_loss: 0.8497 - val_acc: 0.7593
Epoch 3/10
180/180 [==============================] - 82s 456ms/step - loss: 0.1155 - acc: 0.9572 - val_loss: 0.5881 - val_acc: 0.7963
Epoch 4/10
180/180 [==============================] - 81s 452ms/step - loss: 0.0631 - acc: 0.9779 - val_loss: 1.7441 - val_acc: 0.6296
Epoch 5/10
180/180 [==============================] - 81s 451ms/step - loss: 0.0725 - acc: 0.9744 - val_loss: 0.4293 - val_acc: 0.8519
Epoch 6/10
180/180 [==============================] - 81s 452ms/step - loss: 0.0374 - acc: 0.9863 - val_loss: 0.7897 - val_acc: 0.7833
Epoch 7/10
180/180 [==============================] - 81s 451ms/step - loss: 0.0418 - acc: 0.9852 - val_loss: